In [6]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.19.1


In [78]:
train_sents = [[('King' ,'r'), ('Solomon','n' ),('conquered','a'),('France', 'c')],[('Prince','r'),('Charles','n'),('captured','a'),('Rome','c')]]
train_sents

[[('King', 'r'), ('Solomon', 'n'), ('conquered', 'a'), ('France', 'c')],
 [('Prince', 'r'), ('Charles', 'n'), ('captured', 'a'), ('Rome', 'c')]]

In [79]:
test_sents = [[('Queen' ,'r'), ('Elizabeth','n' ),('conquered','a'),('Italy', 'c')],[('Prince','r'),('George','n'),('captured','a'),('Turkey','c')]]
print test_sents

[[('Queen', 'r'), ('Elizabeth', 'n'), ('conquered', 'a'), ('Italy', 'c')], [('Prince', 'r'), ('George', 'n'), ('captured', 'a'), ('Turkey', 'c')]]


In [61]:
person_set = {'Solomon','Charles'}
activity_set = {'conquered','captured'}
place_set = {'France','Rome'}
royalty_set = {'Prince','King'}
presence_of_feature = lambda x,y: x in y

In [64]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'bias',
        #'word.lower=' + word.lower(),
        #'word[-3:]=' + word[-3:],
        #'word[-2:]=' + word[-2:],
        #'word.isupper=%s' % word.isupper(),
        'word.isperson=%s'% presence_of_feature(word,person_set),
        'word.isplace=%s'% presence_of_feature(word,place_set),
        'word.isroyalty=%s'% presence_of_feature(word,royalty_set),
        'word.isactivity=%s'% presence_of_feature(word,activity_set)
        #'word.istitle=%s' % word.istitle(),
        #'word.isdigit=%s' % word.isdigit()
        #'postag=' + postag,
        #'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.isperson=%s'% presence_of_feature(word1,person_set),
            '-1:word.isplace=%s'% presence_of_feature(word1,place_set),
            '-1:word.isroyalty=%s'% presence_of_feature(word1,royalty_set),
            '-1:word.isactivity=%s'% presence_of_feature(word1,activity_set)
            #'-1:word.lower=' + word1.lower(),
            #'-1:word.istitle=%s' % word1.istitle(),
            #'-1:word.isupper=%s' % word1.isupper(),
            #'-1:postag=' + postag1,
            #'-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.isperson=%s'% presence_of_feature(word,person_set),
            '+1:word.isplace=%s'% presence_of_feature(word,place_set),
            '+1:word.isroyalty=%s'% presence_of_feature(word,royalty_set),
            '+1:word.isactivity=%s'% presence_of_feature(word,activity_set)
            #'+1:word.lower=' + word1.lower(),
            #'+1:word.istitle=%s' % word1.istitle(),
            #'+1:word.isupper=%s' % word1.isupper(),
            #'+1:postag=' + postag1,
            #'+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]    

This is what word2features extracts:

In [65]:
y = sent2features(train_sents[0])

In [66]:
y

[['bias',
  'word.isperson=False',
  'word.isplace=False',
  'word.isroyalty=True',
  'word.isactivity=False',
  'BOS',
  '+1:word.isperson=False',
  '+1:word.isplace=False',
  '+1:word.isroyalty=True',
  '+1:word.isactivity=False'],
 ['bias',
  'word.isperson=True',
  'word.isplace=False',
  'word.isroyalty=False',
  'word.isactivity=False',
  '-1:word.isperson=False',
  '-1:word.isplace=False',
  '-1:word.isroyalty=True',
  '-1:word.isactivity=False',
  '+1:word.isperson=True',
  '+1:word.isplace=False',
  '+1:word.isroyalty=False',
  '+1:word.isactivity=False'],
 ['bias',
  'word.isperson=False',
  'word.isplace=False',
  'word.isroyalty=False',
  'word.isactivity=True',
  '-1:word.isperson=True',
  '-1:word.isplace=False',
  '-1:word.isroyalty=False',
  '-1:word.isactivity=False',
  '+1:word.isperson=False',
  '+1:word.isplace=False',
  '+1:word.isroyalty=False',
  '+1:word.isactivity=True'],
 ['bias',
  'word.isperson=False',
  'word.isplace=True',
  'word.isroyalty=False',
  'wor

Extract the features from the data:

In [67]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [80]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [81]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

Possible parameters for the default training algorithm:

Train the model:

In [71]:
trainer.train('test.crfsuite')

In [72]:
print len(trainer.logparser.iterations), trainer.logparser.iterations[-1]

26 {'loss': 9.203317, 'error_norm': 0.00068, 'linesearch_trials': 1, 'active_features': 12, 'num': 26, 'time': 0.0, 'scores': {}, 'linesearch_step': 1.0, 'feature_norm': 2.219804}


In [73]:
tagger = pycrfsuite.Tagger()
tagger.open('test.crfsuite')

Let's tag a sentence to see how it works:

In [83]:
example_sent = test_sents[0]
#print(' '.join(sent2tokens(example_sent)), end='\n\n')
print example_sent
print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

[('Queen', 'r'), ('Elizabeth', 'n'), ('conquered', 'a'), ('Italy', 'c')]
('Predicted:', 'r n a c')
('Correct:  ', 'r n a c')


In [84]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [85]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 151 µs, sys: 4 µs, total: 155 µs
Wall time: 165 µs


..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [86]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

                  1.00      1.00      1.00         8

avg / total       1.00      1.00      1.00         8



## Let's check what classifier learned

In [87]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
a      -> c       1.303724
r      -> n       1.303344
n      -> a       1.049274

Top unlikely transitions:
a      -> c       1.303724
r      -> n       1.303344
n      -> a       1.049274


In [88]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
0.329791 n      -1:word.isroyalty=True
0.210297 c      word.isplace=True
0.210297 c      EOS
0.210297 c      -1:word.isactivity=True
0.210116 r      +1:word.isroyalty=True
0.210116 r      BOS
0.210116 r      word.isroyalty=True
0.164729 a      +1:word.isactivity=True
0.164729 a      word.isactivity=True

Top negative:
0.329791 n      -1:word.isroyalty=True
0.210297 c      word.isplace=True
0.210297 c      EOS
0.210297 c      -1:word.isactivity=True
0.210116 r      +1:word.isroyalty=True
0.210116 r      BOS
0.210116 r      word.isroyalty=True
0.164729 a      +1:word.isactivity=True
0.164729 a      word.isactivity=True
